In [2]:
import pandas as pd
from pyinaturalist import get_observations
import time
import os
import shutil
from datetime import datetime

def agregar_imagenes_a_csv(archivo_csv, fila_inicio=0, hacer_backup=True):
    """
    Añade la columna 'imagen' a un CSV con observaciones de iNaturalist
    
    Args:
        archivo_csv (str): Ruta al archivo CSV de entrada y salida
        fila_inicio (int): Número de fila desde donde comenzar el procesamiento
        hacer_backup (bool): Si es True, crea una copia de seguridad del archivo original
    """
    
    # Crear backup si se solicita
    if hacer_backup:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_file = f"{archivo_csv}.backup_{timestamp}"
        shutil.copy2(archivo_csv, backup_file)
        print(f"Backup creado en: {backup_file}")
    
    try:
        df = pd.read_csv(archivo_csv)
        print(f"CSV cargado: {len(df)} filas encontradas")
    except Exception as e:
        print(f"Error al cargar el CSV: {e}")
        return
    
    if 'imagen' not in df.columns:
        df['imagen'] = None
        print("Columna 'imagen' añadida al DataFrame")
    
    print(f"Comenzando procesamiento desde la fila {fila_inicio}")
    
    for index, row in df.iterrows():
        # Saltar filas hasta llegar a la fila de inicio
        if index < fila_inicio:
            continue
            
        if pd.notna(row.get('imagen')) and row['imagen'] != '':
            print(f"Fila {index}: Ya tiene imagen, saltando...")
            continue
        
        observacion_id = row['id']
        print(f"Procesando fila {index}: ID {observacion_id}")
        
        try:
            response = get_observations(id=observacion_id)
            
            if response['results']:
                observation = response['results'][0]
                
                if observation['photos']:
                    photo_url = observation['photos'][0]['url'].replace('square', 'medium')
                    df.at[index, 'imagen'] = photo_url
                    print(f"  ✓ Imagen encontrada: {photo_url}")
                else:
                    df.at[index, 'imagen'] = 'Sin imagen'
                    print(f"  ✗ Observación sin imágenes")
            else:
                df.at[index, 'imagen'] = 'No encontrada'
                print(f"  ✗ Observación no encontrada")
                
        except Exception as e:
            df.at[index, 'imagen'] = f"Error: {str(e)}"
            print(f"  ! Error al procesar: {e}")
        
        df.to_csv(archivo_csv, index=False)
        print(f"  → Progreso guardado en {archivo_csv}")
        time.sleep(1)
    
    print(f"\nProceso completado! Archivo actualizado en: {archivo_csv}")
    return df

# Uso del código
if __name__ == "__main__":
    archivo_entrada = "ubicaciones.csv"
    
    # Preguntar al usuario desde qué fila quiere empezar
    try:
        fila_inicio = 5002
    except ValueError:
        print("Entrada no válida, comenzando desde la fila 0")
        fila_inicio = 0
    
    # Ejecutar el proceso
    df_resultado = agregar_imagenes_a_csv(archivo_entrada, fila_inicio=fila_inicio, hacer_backup=True)
    
    # Mostrar resumen
    if df_resultado is not None:
        print("\nResumen final:")
        print(f"Total de filas procesadas: {len(df_resultado)}")
        print("Distribución de estados:")
        print(df_resultado['imagen'].value_counts())

Backup creado en: ubicaciones.csv.backup_20251129_180535
CSV cargado: 9999 filas encontradas
Columna 'imagen' añadida al DataFrame
Procesando fila 0: ID 274076247
  ✓ Imagen encontrada: https://inaturalist-open-data.s3.amazonaws.com/photos/492787363/medium.jpeg
  → Progreso guardado en ubicaciones.csv
Procesando fila 1: ID 290035743
  ✓ Imagen encontrada: https://inaturalist-open-data.s3.amazonaws.com/photos/521715680/medium.jpg
  → Progreso guardado en ubicaciones.csv
Procesando fila 2: ID 314934926
  ✓ Imagen encontrada: https://inaturalist-open-data.s3.amazonaws.com/photos/568874088/medium.jpg
  → Progreso guardado en ubicaciones.csv
Procesando fila 3: ID 315164886
  ✓ Imagen encontrada: https://inaturalist-open-data.s3.amazonaws.com/photos/569320745/medium.jpg
  → Progreso guardado en ubicaciones.csv
Procesando fila 4: ID 324187710
  ✓ Imagen encontrada: https://static.inaturalist.org/photos/586719919/medium.jpg
  → Progreso guardado en ubicaciones.csv
Procesando fila 5: ID 2974404

KeyboardInterrupt: 